In [1]:
from tqdm import tqdm
from pprint import pprint
import json
import pandas as pd
import sys
sys.path.append('/home/ajassani/iLoveTrace')
from Trace2Tree.trace_to_tree import TraceToTree
from tree_perf import TreePerfAnalyzer

In [2]:
path = '/home/ajassani/trace_data/jan6/MI325/gpt-3-xl_single-gpu_bfloat16_bs5_steps25_det0_prof1_level1/pytorch_profile_rank0.json'
with open(path, 'r') as f:
    data = json.load(f)

events = data['traceEvents']
tree = TraceToTree(events)
tree.build_tree(add_python_func=False)
perf_analyzer = TreePerfAnalyzer(tree)

Building tree with add_python_func=False
Building CPU op tree with add_python_func=False


In [8]:
df_kernel_launchers = perf_analyzer.get_df_kernel_launchers()
df_kernel_launchers.round(2)

,name,Input Dims,total_direct_kernel_time,direct_kernel_count
0,aten::fill_,"((5,), ())",4.89,1
1,aten::mse_loss_backward,"((), (5,), (5,), (), (5,))",4.77,1
2,aten::copy_,"((5,), (5,), ())",5.25,1
3,aten::div,"((5, 2048, 32), ())",5.57,1
4,aten::mm,"((10240, 32), (32, 2048))",26.80,1
...,...,...,...,...
9994,aten::_foreach_addcmul_,"((), (), (), ())",6033.31,100
9995,aten::_foreach_sqrt,"((),)",3677.09,90
9996,aten::_foreach_div_,"((), ())",4721.54,82
9997,aten::_foreach_add_,"((), ())",4687.17,81


In [7]:
# this gives a good perf breakdown at op level
df_kernel_launchers_summary = perf_analyzer.get_df_kernel_launchers_summary(df_kernel_launchers)
df_kernel_launchers_summary.round(2)

,name,total_direct_kernel_time_sum,Count,total_direct_kernel_time_ms,Percentage (%),Cumulative Percentage (%)
13,aten::convolution_backward,363535.30,736,363.54,34.05,34.05
20,aten::miopen_convolution,156527.22,736,156.53,14.66,48.71
21,aten::mm,84654.09,300,84.65,7.93,56.64
19,aten::miopen_batch_norm_backward,59738.87,448,59.74,5.60,62.23
33,flash_attn::_flash_attn_backward,54526.56,25,54.53,5.11,67.34
9,aten::addmm,45762.04,147,45.76,4.29,71.62
18,aten::miopen_batch_norm,43081.60,448,43.08,4.03,75.66
12,aten::clamp_min_,34283.57,416,34.28,3.21,78.87
32,aten::threshold_backward,31333.86,576,31.33,2.93,81.81
14,aten::copy_,30761.00,2858,30.76,2.88,84.69


In [9]:
# We can further get breakdown by shapes for a particular op
df_kernel_launchers_summary_name_shapes = perf_analyzer.get_df_kernel_launchers_summary(df_kernel_launchers, filter_names=['aten::clamp_min_'], group_by_shape=True)
df_kernel_launchers_summary_name_shapes.round(2)

,name,Input Dims,total_direct_kernel_time_sum,Count,total_direct_kernel_time_ms,Percentage (%),Cumulative Percentage (%)
5,aten::clamp_min_,"((5, 896, 59, 91), ())",10420.67,232,10.42,30.40,30.40
3,aten::clamp_min_,"((5, 448, 118, 182), ())",7888.23,88,7.89,23.01,53.40
1,aten::clamp_min_,"((5, 224, 235, 363), ())",6288.14,40,6.29,18.34,71.75
2,aten::clamp_min_,"((5, 224, 470, 725), ())",4172.48,8,4.17,12.17,83.92
4,aten::clamp_min_,"((5, 448, 235, 363), ())",2193.02,8,2.19,6.40,90.31
6,aten::clamp_min_,"((5, 896, 118, 182), ())",1286.46,8,1.29,3.75,94.07
0,aten::clamp_min_,"((5, 32, 470, 725), ())",813.62,8,0.81,2.37,96.44
8,aten::clamp_min_,"((5, 2016, 59, 91), ())",786.06,8,0.79,2.29,98.73
7,aten::clamp_min_,"((5, 2016, 30, 46), ())",434.89,16,0.43,1.27,100.00
